# Define all constants corresponding to game data

Define resources

In [80]:
RESOURCES = [
    "VP", # Victory points
    "Q", # Quest cards (as rewards to be obtained)
    # "I", # Intrigue cards (as rewards to be obtained) # TODO (later versions): Uncomment this
    "Purple", # Wizard item, i.e. purple cubes
    "White", # Cleric item, i.e. white cubes
    "Black", # Rogue item, i.e. black cubes
    "Orange", # Fighter item, i.e. orange cubes
    "Gold", # Gold
]

Define all quests in the game

In [56]:
QUEST_TYPES = ["Arcana", "Piety", "Skullduggery", "Warfare", "Commerce"]

In [57]:
from collections import namedtuple
Quest = namedtuple("Quest", "name type requirements rewards")

QUESTS = [
    Quest("Simple Warfare", "Warfare", {"Orange": 2}, {"VP": 4}),
    Quest("Simple Skullduggery", "Skullduggery", {"Black": 2}, {"VP": 4}),
    Quest("Simple Piety", "Piety", {"White": 1}, {"VP": 4}),
    Quest("Simple Arcana", "Arcana", {"Purple": 1}, {"VP": 4}),
    Quest("Simple Commerce", "Commerce", {"Gold": 4}, {"VP": 4}),
]

Verify correctness of quest attributes

In [58]:
for quest in QUESTS:
    for rewardKey in quest.rewards:
        assert rewardKey in RESOURCES, quest
    for requirementKey in quest.requirements:
        assert requirementKey in RESOURCES, quest
    assert quest.type in QUEST_TYPES

Demonstrate how to access quest data

In [59]:
orangeQuest = QUESTS[0]
print(orangeQuest)
print(orangeQuest.type)
print(orangeQuest.rewards)
print(orangeQuest.rewards["VP"])

Quest(name='Simple Warfare', type='Warfare', requirements={'Orange': 2}, rewards={'VP': 4})
Warfare
{'VP': 4}
4


Demonstrate that quests are immutable

In [60]:
orangeQuest.rewards = {"black": 2} # Results in AttributeError

AttributeError: can't set attribute

Define number of agents per player as a function of number of players

In [81]:
def agentsPerPlayer(numPlayers: int):
    if numPlayers == 2: return 4
    if numPlayers == 3: return 3
    if numPlayers == 4: return 2
    if numPlayers == 5: return 2 
    else: 
        raise ValueError("Number of players must be an integer between 2 and 5, inclusive.")

# Create classes for the players and the game state

In [28]:
import random 

In [82]:
class Player():
    def __init__(self, name: str, numAgents: int) -> None:
        '''Initialize the player's name, resources, agents, and VPs.'''
        self.name = name 
        self.resources = {
            "Purple": 0,
            "White": 0,
            "Black": 0,
            "Orange": 0,
            "Gold": 0
        }
        for resource in self.resources:
            assert resource in RESOURCES

        self.activeQuests = []
        self.completedQuests = []
        # self.plotQuests = [] # Completed plot quests
        # self.intrigues = []
        self.VP = 0
        self.agents = numAgents
        self.maxAgents = numAgents

    def receiveQuest(self, quest: Quest):
        '''
        Receive a quest.

        Args: 
            quest: the quest to receive.
        '''
        self.activeQuests.append(quest)

    # TODO (Later version): uncomment this
    # def receiveIntrigue(self, intrigue: Intrigue):
        # '''
        # Receive an intrigue card.

        # Args: 
        #     intrigue: the intrigue card to receive.
        # '''
    #     self.intrigues.append(intrigue)

    def receiveResource(self, resource: str, number: int):
        '''
        Receive some number of resources of the same type.

        Args: 
            resource: the type of resource.
            number: the number of that resource type received.
        '''
        if resource not in RESOURCES:
            raise ValueError("Invalid resource type.")
        if resource in ["Q", "I"]:
            raise ValueError("Cannot receive quests or intrigue cards with this function. \
                             Use 'receiveQuest' or 'receive Intrigue', respectively.")
        if number <= 0:
            raise ValueError("Cannot receive nonnegative resource count.")
        
        self.resources[resource] += number
        
    # TODO: This really should be in GameState
    # def completeQuest(self, quest: Quest):
    #     # Check if the quest can be completed
    #     validCompletion = True
    #     for resource in quest.requirements:
    #         if quest.requirements[resource] > self.resources[resource]:
    #             validCompletion = False
                
    #     if validCompletion:
    #         # TODO: Finish this
    #         for resource in quest.requirements:
    #             self.resources[resource] -= quest.requirements[resource]
    #             self.
    #     else:
    #         raise ValueError("Do not have enough resources to complete this quest.")
        
    #     # Check that all resource counts are still nonnegative
    #     for resourceNumber in self.resources.values():
    #         assert resourceNumber >= 0
        

In [85]:
class GameState():
    def __init__(self, numPlayers: int = 3, numRounds: int = 8, 
                 playerNames = None) -> None:
        '''
        Initialize the game state. Creates the quest stack,
        initializes all buildings states, ... (MORE TODO HERE)

        Args: 
            numPlayers: the number of players in the game
            numRounds: the number of rounds in the game
            playerNames (optional): the names for each player
        '''
        # Initialize the remaining number of rounds
        self.roundsLeft = numRounds

        # Check that we have a valid number of players
        assert numPlayers >= 2 and numPlayers <= 5

        # Initialize the players
        self.playerNames = playerNames
        if playerNames == None:
            self.playerNames = [
                "PlayerOne", "PlayerTwo", 
                "PlayerThree", "PlayerFour",
                "PlayerFive"
            ][:numPlayers]
        
        self.players = [Player(self.playerNames[i], agentsPerPlayer(numPlayers)) for i in range(numPlayers)]
            
        # Create the quest stack
        self.questStack = QUESTS.copy()
        random.shuffle(self.questStack)

        # Initialize all building states
        # TODO (in later versions): have all 3 Cliffwatch Inn spots instead of just 1
        # TODO (in later versions): uncomment the commented ones below
        self.purpleOccupied = False 
        self.buildingStates = { # True if the building is occupied
            "Purple": False, # Blackstaff Tower (for Wizards)
            "Orange": False, # Field of Triumph (for Fighters)
            "White": False, # The Plinth (for Clerics)
            "Black": False, # The Grinning Lion Tavern (for Rogues)
            "Gold": False, # Aurora's Realms Shop (for Gold)
            "Quest": False, # Cliffwatch Inn (for Quests)
            # "Castle": False # Castle Waterdeep (for Castle + Intrigue)
            # "Builder": False # Builder's Hall (for buying Buildings)
            # "Waterdeep1": False # Waterdeep Harbor, first slot (for playing Intrigue)
            # "Waterdeep2": False # Waterdeep Harbor, second slot (for playing Intrigue)
            # "Waterdeep3": False # Waterdeep Harbor, third slot (for playing Intrigue)
        }

        # Deal quest cards to players
        for _ in range(2):
            for player in self.players:
                player.receive(self.questStack.pop())

        # TODO (later version): Deal intrigue cards to players

        # Finally, start a new round (at this stage, just 
        # decrements roundsLeft and places VPs on
        # buildings at builder's hall)
        self.newRound()

    def newRound(self) -> None:
        '''Reset the board at the beginning of each round.'''
        self.roundsLeft -= 1
        # TODO (later version): put VPs on buildings at bulider's hall

        # Reset all buildings
        for building in self.buildingStates:
            self.buildingStates[building] = False

        # TODO (later version): put new resources on buildings that need them

        # Get new agent at fifth round
        if self.roundsLeft == 4:
            for player in self.Players:
                player.maxAgents += 1

        # Return all agents
        for player in self.Players:
            player.agents = player.maxAgents

    
    def drawQuest(self) -> Quest:
        '''
        Draw the top quest from the quest stack,
        removing it from the stack in the process.
        
        Returns: 
            The top quest from the quest stack.
        '''
        return self.questStack.pop()
    
    def printQuestStack(self) -> None:
        '''Debug function for printing the quest stack.'''
        print("Quest stack (top first):")
        questStackCopy = self.questStack.copy()
        for i in range(len(self.questStack)):
            print(i+1, questStackCopy.pop())

    def displayGameState(self) -> None:
        '''Display the state of the game.'''
        # TODO: Implement this to display the state of the game
        # Will start textually, should be graphically later
        pass

Test the quest stack

In [79]:
gameState = GameState()

for i in range(2):
    print("Drew quest:", gameState.drawQuest())
    gameState.printQuestStack()
    print()

Drew quest: Quest(name='Simple Commerce', type='Commerce', requirements={'Gold': 4}, rewards={'VP': 4})
Quest stack (top first):
1 Quest(name='Simple Arcana', type='Arcana', requirements={'Purple': 1}, rewards={'VP': 4})
2 Quest(name='Simple Warfare', type='Warfare', requirements={'Orange': 2}, rewards={'VP': 4})
3 Quest(name='Simple Skullduggery', type='Skullduggery', requirements={'Black': 2}, rewards={'VP': 4})
4 Quest(name='Simple Piety', type='Piety', requirements={'White': 1}, rewards={'VP': 4})

Drew quest: Quest(name='Simple Arcana', type='Arcana', requirements={'Purple': 1}, rewards={'VP': 4})
Quest stack (top first):
1 Quest(name='Simple Warfare', type='Warfare', requirements={'Orange': 2}, rewards={'VP': 4})
2 Quest(name='Simple Skullduggery', type='Skullduggery', requirements={'Black': 2}, rewards={'VP': 4})
3 Quest(name='Simple Piety', type='Piety', requirements={'White': 1}, rewards={'VP': 4})

